In [1]:
import pandas as pd
combined_dataset = pd.read_pickle('../combined_dataset_notes.pkl')

In [6]:

combined_dataset = combined_dataset.drop(['HADM_ID','ADMISSION_DAYS'],axis=1)
print(combined_dataset.shape)
combined_dataset.head()

(45526, 503)


,text,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
47366,"[chest, pa, lat, clip, clip, number, radiology...",0,35,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,0,1,0,0,0,0,0,0,0,1
47024,"[pm, liver, gallbladder, u, single, organ, cli...",1,60,462.0,-6.0,24.0,17.0,1.0,104.0,1.06,...,0,1,0,0,0,1,0,0,0,0
9046,"[chest, portable, ap, clip, clip, number, radi...",0,49,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,0,0,0,0,0,1,0,0,0,1
843,"[pm, chest, pa, lat, clip, clip, number, radio...",1,61,462.0,2.0,24.0,28.0,1.0,103.0,1.26,...,0,0,0,0,0,0,0,1,0,1
43466,"[pm, chest, portable, ap, different, physician...",0,54,462.0,0.0,23.0,25.0,1.0,106.0,1.12,...,0,0,0,0,0,0,0,0,0,0


In [7]:
only_text = combined_dataset[["text","G1","G2","G3","G4","G5","G6","G7","G8","G9","G10","G11","G12","G13","G14","G15","G16","G17","G18","G19","G20"]]
only_text.shape

(45526, 21)

In [7]:
!pip install --upgrade gensim
!pip install -U gensim

Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)


In [92]:

from gensim.models import Word2Vec

In [249]:
#pass text here..
model = Word2Vec(combined_dataset.text,size=1000, window=10, min_count=5, workers=56)#play around with param
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
#print(model['sentence'])
# save model
model.save('model1000-10-5.bin')
# load model
new_model = Word2Vec.load('model1000-10-5.bin')
print(new_model)

Word2Vec(vocab=23960, size=1000, alpha=0.025)
['chest', 'pa', 'lat', 'clip', 'number', 'radiology', 'reason', 'eval', 'infiltrate', 'hospital', 'medical', 'condition', 'year', 'old', 'woman', 'vomiting', 'examination', 'final', 'report', 'radiograph', 'indication', 'evaluation', 'infiltrates', 'comparison', 'finding', 'compared', 'previous', 'relevant', 'change', 'evidence', 'pneumonia', 'pathological', 'abnormality', 'pleural', 'effusion', 'pulmonary', 'edema', 'normal', 'size', 'cardiac', 'silhouette', 'pm', 'liver', 'gallbladder', 'u', 'single', 'organ', 'portal', 'vein', 'thrombosis', 'tapable', 'ascites', 'please', 'mark', 'man', 'disease', 'hypertension', 'worsening', 'lfts', 'wet', 'read', 'gwp', 'fri', 'pv', 'patent', 'hepatopetal', 'flow', 'query', 'lliver', 'ultrasound', 'doctor', 'last', 'name', 'scale', 'color', 'doppler', 'son', 'ni', 'image', 'demonstrate', 'persistently', 'coarsened', 'echotexture', 'nodular', 'surface', 'consistent', 'known', 'history', 'cirrhosis', 'la

Word2Vec(vocab=23960, size=1000, alpha=0.025)


In [94]:
y = only_text.iloc[:,1:]
x = only_text.iloc[:,:1]

In [97]:
y.shape

(45526, 20)

In [101]:
from sklearn.model_selection import train_test_split
seed=243 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(38697, 1) (6829, 1) (38697, 20) (6829, 20)


In [9]:
'''%matplotlib inline
from sklearn.decomposition import PCA
from matplotlib import pyplot
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()'''

'%matplotlib inline\nfrom sklearn.decomposition import PCA\nfrom matplotlib import pyplot\nX = model[model.wv.vocab]\npca = PCA(n_components=2)\nresult = pca.fit_transform(X)\n# create a scatter plot of the projection\npyplot.scatter(result[:, 0], result[:, 1])\nwords = list(model.wv.vocab)\nfor i, word in enumerate(words):\n    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))\npyplot.show()'

In [10]:
#voila

In [11]:
#avg_vec.shape

In [99]:
WordVectorz=dict(zip(model.wv.index2word,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz)

In [100]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss

#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=1000,
#                                                                     learning_rate=0.15))

#from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import numpy as np
pipe1=Pipeline([("wordVectz",AverageEmbeddingVectorizer(WordVectorz)),("multilabel",OneVsRestClassifier(LGBMClassifier(iterations=1000,learning_rate=0.15,n_jobs=-1)))])
pipe1.fit(x_train.text,y_train)
# clf_multilabel.fit(x_train.text,y_train)#have to now oopsitsk


In [172]:
predicted1=pipe1.predict(x_test.text)

In [29]:
predicted1_prob_train = pipe1.predict_proba(x_train.text)
predicted1_prob_test = pipe1.predict_proba(x_test.text)

In [173]:
print(jaccard_score(y_test,predicted1,average='micro'))
print(jaccard_score(y_test,predicted1,average='macro'))
print(roc_auc_score(y_test,predicted1))
print(hamming_loss(y_test,predicted1))
'''
Cat w2v 500,10,5
0.6633380308436122
0.18253038512227265'''

0.5034983002608902
0.39044713301507433
0.6646507472338611
0.1839288329184361


'\nCat w2v 500,10,5\n0.6633380308436122\n0.18253038512227265'

In [89]:
predicted1.shape #earlier with 1500d & mincount=3 - 0.6573500885474709

(6829, 19)

In [44]:
#pipe1.fit(x_train.text,y_train)
predicted1_prob_testdf = pd.DataFrame(predicted1_prob_test)
predicted1_prob_traindf = pd.DataFrame(predicted1_prob_train)

In [45]:
predicted1_prob_test.shape

(6829, 20)

In [46]:
predicted1_prob_testdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.006364,0.014811,0.381461,0.244952,0.226117,0.133170,0.422299,0.114607,0.154242,0.135912,0.000083,0.029295,0.206063,0.008920,0.000142,0.144964,0.027488,0.984861,0.983045,0.347243
1,0.060448,0.020596,0.792334,0.469381,0.554800,0.337727,0.471109,0.146749,0.755750,0.457869,0.001473,0.103819,0.162318,0.017451,0.000144,0.527361,0.014730,0.352813,0.349358,0.289949
2,0.011747,0.047712,0.474600,0.111793,0.401464,0.651509,0.855404,0.151393,0.127526,0.281706,0.000056,0.029521,0.174612,0.006954,0.000131,0.310654,0.001712,0.515857,0.806501,0.454475
3,0.009529,0.030799,0.778939,0.136471,0.182365,0.104084,0.989231,0.141261,0.191664,0.090097,0.000048,0.020778,0.120590,0.015656,0.000347,0.082227,0.001190,0.168287,0.069491,0.452310
4,0.155675,0.350358,0.850952,0.722703,0.364650,0.714532,0.905861,0.852393,0.701567,0.771462,0.001481,0.397246,0.220130,0.035454,0.000101,0.672055,0.041518,0.754205,0.359311,0.675329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6824,0.095784,0.137530,0.860846,0.712012,0.310655,0.738019,0.927311,0.315240,0.471653,0.871125,0.000228,0.494851,0.312031,0.008207,0.000102,0.528199,0.029597,0.859444,0.631167,0.771659
6825,0.070175,0.068985,0.846435,0.632877,0.799036,0.569135,0.780485,0.465946,0.804712,0.705267,0.001412,0.456766,0.196635,0.019320,0.000352,0.601936,0.041650,0.657041,0.343640,0.647979
6826,0.042937,0.040011,0.640910,0.506048,0.862476,0.538513,0.452046,0.583061,0.328257,0.358777,0.000726,0.209272,0.294012,0.011878,0.000791,0.636439,0.031056,0.493248,0.481291,0.412246
6827,0.105662,0.080924,0.846555,0.791480,0.317569,0.542417,0.959383,0.258478,0.417617,0.915276,0.000217,0.367771,0.196251,0.009987,0.000130,0.452880,0.026391,0.855679,0.562148,0.643258


x_train.text.to_pickle('./train-nlp.pkl')
x_test.text.to_pickle('./test-nlp.pkl')
y_train.to_pickle('./y_train-nlp.pkl')
y_test.to_pickle('./y_test-nlp.pkl')
predicted1_prob_testdf.to_pickle('./pred-test-xgb.pkl')
predicted1_prob_traindf.to_pickle('./pred-train-xgb.pkl')

In [174]:
from sklearn.externals import joblib
joblib.dump(pipe1, "w2v_lgbm.pkl")

['w2v_lgbm.pkl']

In [175]:
textlgbm =  joblib.load("w2v_lgbm.pkl")


In [151]:
valc = joblib.load("cat1-C.pkl")
textc = joblib.load("./w2v.pkl")
lgbm = joblib.load("./LGBM1-C.pkl")

KeyboardInterrupt: 

In [9]:
x_full = combined_dataset.iloc[:,:483]
y_full = combined_dataset.iloc[:,483:]

In [11]:
y_full.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [12]:
from sklearn.model_selection import train_test_split
seed=243 
x_train_full,x_test_full,y_train_full,y_test_full = train_test_split(x_full,y_full,test_size=0.15,random_state=seed)
x_test_full.iloc[:,1:483]

,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,50809,...,51512,51513,51514,51515,51516,51517,51518,51519,51529,51532
21738,1,83,462.0,5.0,24.0,32.0,1.0,104.0,1.12,145.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
47854,1,66,462.0,-2.0,22.0,22.0,1.0,105.0,1.10,146.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
3615,1,80,462.0,1.0,24.0,26.0,1.0,104.0,1.15,120.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
37273,1,46,462.0,0.0,24.0,26.0,1.0,104.0,1.12,127.0,...,1.0,1.044,4.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
12132,1,85,338.0,-1.0,24.0,25.0,1.0,100.0,1.14,268.0,...,1.0,1.044,0.2,1.0,0.0,3.0,0.0,0.0,135.5,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542,1,82,584.0,0.0,24.0,26.0,1.0,104.0,1.12,127.0,...,1.0,1.044,0.2,1.0,7.0,3.0,0.0,0.0,135.5,43.0
57534,1,56,462.0,0.0,24.0,29.0,1.0,102.0,1.25,86.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
40358,1,36,462.0,0.0,24.0,26.0,1.0,104.0,1.12,127.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
24988,1,50,391.0,6.0,24.0,32.0,1.0,101.0,1.18,131.0,...,1.0,1.044,0.2,1.0,0.0,3.0,0.0,0.0,135.5,43.0


In [111]:
y_pred_catnum = valc.predict(x_test_full.iloc[:,1:483])
y_pred_catext = textc.predict(x_test_full.text)

print(y_pred_catnum.shape,y_pred_catext.shape)

(6829, 20) (6829, 20)


In [209]:
y_pred_lg = lgbm.predict(x_test_full.iloc[:,1:483])
y_pred_lg_prob = lgbm.predict_proba(x_test_full.iloc[:,1:483])
print(y_pred_lg.shape)

(6829, 20)


In [203]:
y_pred_lgbm_text = textlgbm.predict(x_test_full.text)
y_pred_lgbm_text_prob = textlgbm.predict_proba(x_test_full.text)

print(y_pred_lgbm_text.shape)

(6829, 20)


In [120]:
y_pred_catnum_prob = valc.predict_proba(x_test_full.iloc[:,1:483])
y_pred_catext_prob = textc.predict_proba(x_test_full.text)
print(y_pred_catnum.shape,y_pred_catext.shape)

(6829, 20) (6829, 20)


In [239]:
y_comb =y_pred_catext_prob
#y_comb/=2
y_comb = y_comb.round()
#just try this, according to me text can give more info, and i would change w2v to 1000 
# 0.7052650131769738
# 0.1599868209108215

In [248]:
# predicting on avg of both probs
print(jaccard_score(y_test,y_comb,average='micro'))
print(jaccard_score(y_test,y_comb,average='macro'))
print(roc_auc_score(y_test,y_comb))
print(hamming_loss(y_test,y_comb))
#0.6646507472338611

0.5893611732537907
0.4653835393750866
0.7067343115696783
0.1572411773319666


In [247]:
auc_sum = 0.17367843022404453**0.7668781779155269 + 0.18253038512227265**0.6633380308436122
#auc/ham worked...
y_comb = (y_pred_lg_prob*0.17367843022404453**0.7668781779155269 + y_pred_catext_prob*0.18253038512227265**0.6633380308436122)/auc_sum
y_comb = y_comb.round()

In [222]:
import numpy as np
def corr(a, b):
    counts = 0
  #first_df = pd.read_csv(first_file,index_col=0)
  #second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
    for i in range(a.shape[0]):  
      #prediction = a[:,i] #first_df.columns[0]
      # correlation
        #print("Finding correlation row : {}".format(i))
      #print("Column to be measured: {}".format(prediction))
        cor = np.corrcoef(a[i,:],b[i,:])
        #print("Pearson's correlation score: {}".format(cor))
        if(cor[0][1] < 0.7):
            counts+=1
      #print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
      #print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))
    return counts

In [223]:
l = [corr(y_pred_catnum,y_pred_lg)]
l# this was high thats y its working better ig
#what was high? like correlation was low for many inspite o
# 4717 cat v cat
# 4909 lgbm vs cat

[521]

In [ ]:
estimators=[ ('text', textc), ('val', valc)]
ensemble = OneVsRestClassifier(VotingClassifier(estimators, voting='soft'))
ensemble.fit(x_train_mm, y_train)

In [ ]:
# I'll explain, firstly i tweaked params of w2v slightly..and used catboost gave arnd 
# then i used catboost of values and took average of probs, 1 sec ill go through and ill give u a gist
#in that way its easier to correct me...ok